# Convolutional Neural Network

해당 repository 및 code들은 wikidocs에서 제공하는 'Pytorch로 시작하는 딥 러닝 입문'을 참고하였음을 밝힙니다. 해당 자료를 바탕으로 숙지한 개념과 추가적인 저의 생각을 기록할 계획입니다. 출처는 다음 사이트와 같습니다.<br><br>
* https://wikidocs.net/book/2788

# 01. 합성곱과 풀링(Convolution and Pooling)

# 02. CNN으로 MNIST 분류하기

### 1. 모델 이해하기

```
# 1번 레이어: 합성곱층(Convolutional layer)
합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU
맥스풀링(kernel_size=2, stride=2)

# 2번 레이어: 합성곱층(Convolutional layer)
합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU
맥스풀링(kernel_size=2, stride=2)

# 3번 레이어: 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size x 7 x 7 x 64 --> batch_size x 3136
전결합층(뉴런 10개) + 활성화함수 Softmax


```

### 2. 모델 구현하기

In [1]:
import torch
import torch.nn as nn

In [2]:
# 배치 크기 x 채널 x 높이(height) x 너비(width)의 크기의 텐서를 선언

inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기: {}'.format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [3]:
# 첫 번째 합성곱 층

conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
# 두 번째 합성곱 층

conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
# 맥스풀링

pool = nn.MaxPool2d(2)  # 정수 하나를 인자로 넣으면 kernel size와 stride가 둘 다 해당값으로 지정
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
out = conv1(inputs)     # inputs.shape=(1, 1, 28, 28)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [8]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [9]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [10]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [11]:
# n번째 차원을 접근하게 해주는 .size(n)
# 첫 번째 차원

out.size(0)

1

In [12]:
# 두 번째 차원

out.size(1)

64

In [13]:
# 세 번째 차원

out.size(2)

7

In [14]:
# 네 번째 차원

out.size(3)

7

In [15]:
# 첫 번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라

out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [16]:
fc = nn.Linear(3136, 10)    # input_dim=3136, output_dim=10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


### 3. CNN으로 MNIST 분류하기

In [18]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [19]:
# 만약 GPU를 사용 가능하다면 device값이 cuda가 되고, 아니라면 cpu가 됩니다.

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [20]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [21]:
mnist_train = dsets.MNIST(root='MNIST_data/',               # 다운로드 경로 지정
                          train=True,                       # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(),  # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',                # 다운로드 경로 지정
                         train=False,                       # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(),   # 텐서로 변환
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [23]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [24]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIN shape= (?, 28, 28, 1)
        #   Conv    -> (?, 28, 28, 32)
        #   Pool    -> (?, 14, 14, 32)
        self.layer1= torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 두번째층
        # ImgIn shape= (?, 14, 14, 32)
        #   Conv    -> (?, 14, 14, 64)
        #   Pool    -> (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)     # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [25]:
# CNN 모델 정의

model = CNN().to(device)

In [27]:
# 비용 함수와 옵티마이저 정의

criterion = torch.nn.CrossEntropyLoss().to(device)  # 비용 함수에 소프트맥스 함수 포함되어져 있음
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [28]:
# 총 배치의 수 출력
# 총 배치의 수 600 x 배치 크기 100 = 총 훈련 데이터 60000

total_batch = len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [29]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:    # 미니 배치 단위로 꺼내온다. X는 미니배치, Y는 레이블
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch:    1] cost = 0.209922969
[Epoch:    2] cost = 0.0634482503
[Epoch:    3] cost = 0.0452167392
[Epoch:    4] cost = 0.0373480245
[Epoch:    5] cost = 0.0280006379
[Epoch:    6] cost = 0.0251064617
[Epoch:    7] cost = 0.0203426927
[Epoch:    8] cost = 0.017274484
[Epoch:    9] cost = 0.0139225516
[Epoch:   10] cost = 0.0122496476
[Epoch:   11] cost = 0.0114028519
[Epoch:   12] cost = 0.00880837254
[Epoch:   13] cost = 0.00778207602
[Epoch:   14] cost = 0.00702229235
[Epoch:   15] cost = 0.00651085889


In [30]:
#  학습을 진행하지 않을 것이므로 torch.no_grad()

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9855999946594238


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
